In [10]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2

from sklearn.utils import shuffle

In [11]:
def load_samples(csv_filename):
    data = pd.read_csv(os.path.join('data_files',csv_filename))
    data = data[['FaceFileName','FlowFileName','Label','ClassName']]
    face_file_names = list(data.iloc[:,0])
    flow_file_names = list(data.iloc[:,1])
    labels = list(data.iloc[:,2])
    samples =[]
    for face,flow,lab in zip(face_file_names,flow_file_names,labels):
        samples.append([face,flow,lab])
    
    return samples

In [12]:
data_path = "train.csv"

In [13]:
samples = load_samples(data_path)

In [14]:
len(samples)

542997

In [15]:
def shuffle_data(data):
    data = shuffle(data)#,random_state=2)
    return data

In [16]:
#shuffle_data(samples)

### we need a bit of more preparation like normalizing the data and converting the labels to one hot encoding

In [17]:
def preprocessing(face_img,flow_img,label):
    face_img = face_img/255
    flow_img = flow_img/255
    num_classes = 2
    one_hot_label = np.array([0,0])
    one_hot_label[label] = 1
    
    return face_img,flow_img,one_hot_label

In [18]:
def data_generator(samples,batch_size=32,shuffle_data = True):
    num_samples = len(samples)
    while True:
        if shuffle_data:
            samples = shuffle(samples)
        
        for offset in range(0,num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            face_train = []
            flow_train = []
            y_train = []
            
            for batch_sample in batch_samples:
                face_img_name = batch_sample[0]
                flow_img_name = batch_sample[1]
                label = batch_sample[2]
                face_img =  cv2.imread(face_img_name)
                flow_img = cv2.imread(flow_img_name)
                
                face_img,flow_img,label = preprocessing(face_img,flow_img,label)
                
                face_train.append(face_img)
                flow_train.append(flow_img)
                y_train.append(label)
                
            face_train = np.array(face_train)
            flow_train = np.array(flow_train)
            
            y_train = np.array(y_train)

            yield [face_train,flow_train],y_train

In [19]:
test_datagen = data_generator(samples,batch_size = 8)

In [20]:
x,y = next(test_datagen)
print(len(x))
print(len(y))
print(y)

2
8
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [21]:
x[0].shape

(8, 50, 50, 3)

In [22]:
x[1].shape

(8, 50, 50, 3)

In [23]:
# for i in range(8):
#     plt.imshow(x[0][i])
#     plt.title(f"class_name: {y[i]}")
#     plt.show()
    

In [24]:
# for i in range(8):
#     plt.imshow(x[1][i])
#     plt.title(f"class_name: {y[i]}")
#     plt.show()

In [25]:
train_data_path = 'train.csv'
test_data_path = 'test.csv'

train_samples = load_samples(train_data_path)
test_samples = load_samples(test_data_path)

num_train_samples = len(train_samples)
num_test_samples = len(test_samples)

print('number of train samples: ', num_train_samples)
print('number of test samples: ', num_test_samples)

number of train samples:  542997
number of test samples:  232526


In [26]:
train_generator = data_generator(train_samples)

In [27]:
x,y = next(train_generator)

In [28]:
x[0].shape


(32, 50, 50, 3)

In [29]:
x[1].shape

(32, 50, 50, 3)

In [30]:
#print(x[0])

### Testing on a Random Model

In [53]:
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras import Model

In [54]:
def create_convolution_layers(input_img):
  model = Conv2D(32, (3, 3), padding='same', input_shape=(50,50,3))(input_img)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D((2, 2),padding='same')(model)
  model = Dropout(0.25)(model)
  
  model = Conv2D(64, (3, 3), padding='same')(model)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
  model = Dropout(0.25)(model)
    
  model = Conv2D(128, (3, 3), padding='same')(model)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
  model = Dropout(0.4)(model)
    
  return model

In [55]:
face_input = Input(shape = (50,50,3))
face_model = create_convolution_layers(face_input)


flow_input = Input(shape = (50,50,3))
flow_model = create_convolution_layers(flow_input)

conv = Concatenate()([face_model,flow_model])

conv = Flatten()(conv)

dense = Dense(512)(conv)

dense = LeakyReLU(alpha=0.1)(dense)
dense = Dropout(0.5)(dense)

output = Dense(2, activation='softmax')(dense)

model = Model(inputs=[face_input, flow_input], outputs=[output])


model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
batch_size = 32
train_generator = data_generator(train_samples,batch_size = 32)

model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs = 2,
)